<a href="https://colab.research.google.com/github/yota-nagai/combinatorial-optimization-problems/blob/main/07_%E3%82%AF%E3%83%AA%E3%83%BC%E3%82%AF%E3%82%AB%E3%83%90%E3%83%BC%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q amplify
from amplify import VariableGenerator, FixstarsClient, solve, Solver
from datetime import timedelta

client = FixstarsClient()
client.token = "******************"  #アクセストークンを入力
client.parameters.timeout = timedelta(milliseconds=2000)  # タイムアウトは 2000 ms


#################### クリークカバー問題 ####################

n=int(input()) #与えられる色の個数　※使われない色があってもok
N,M=map(int,input().split())  #頂点数、辺の数
edge=[]  #辺リスト（無向グラフ）
for _ in range(M):
  u,v=map(int,input().split())
  u-=1
  v-=1
  edge.append([u,v])
  # [v,u]を加えてはダメ　全頂点ペアではなく、辺リストを巡回する

#バイナリ変数の生成
gen=VariableGenerator()
q=gen.array("Binary",shape=(N,n))  #頂点vが色iで塗られているか

#制約項①　各頂点の色はただ1つに決定
f1=0
for v in range(N):
  cnt=0
  for i in range(n):
    cnt+=q[v][i]
  f1+=(cnt-1)**2

#制約項②　同じ色の頂点グループがクリークをなす
f2=0
for i in range(n):
  num_exist=0
  for u,v in edge:
    num_exist+=q[u][i]*q[v][i]
  cnt=0
  for v in range(N):
    cnt+=q[v][i]
  required=cnt*(cnt-1)/2
  f2+=required-num_exist

#ペナルティ係数を設定
A,B=map(int,input().split())
f=A*f1+B*f2


#ソルバー実行、最適解の取得
result=solve(f,client)
objective=result.best.objective
values=list(result.best.values.values())
#print(values)

#エネルギー関数を最小にする塗分け方
print(f"エネルギー関数の最小値 = {objective}")
group=[[] for _ in range(n)]
for v in range(N):
  for i in range(n):
    if values[v*n+i]==True:
      group[i].append(v+1)
print(*group)
